In [ ]:
import pandas as pd
import re

# Read the Excel file into a pandas DataFrame
#skiprows=range(1, 5006) nrows=6
df = pd.read_excel(r'C:\codenrock\test_data_new.xlsx')

# Define the punctuations to remove
punctuations = '''!()«»-[];:'"\,<>/?@#$%^&*_~\t'''

# Define the words to remove
words_to_remove = ['NAME', 'EMAIL', 'PHONE', 
                   'Пожалуйста не изменяйте адрес получателя при ответе на это письмо', 'Аэроклуб', 
                   'Консультант Отдел по обслуживанию ключевых клиентов компании Аэроклуб', 
                   'Россия 107113', 'г. Москва 3я Рыбинская ул. д. 18 стр. 22',
                   'Бизнес  центр',  'Бизнес центр Буревестник',   
                   'вход со 2й Рыбинской ул' ]

# Define the phrases after which text should be deleted
phrases_to_remove_after = ['Данное сообщение с вложениями может', 
                           'если у Вас возникли отзывы о нашей работе', 
                           'The information contained in this email',
                           'may contain confidential',
                           'ОГОВОРКА ОБ ОГРАНИЧЕНИИ ОТВЕТСТВЕННОСТИ',
                           'and its attachments may contain',
                           'Билет на поездку в вагоне']


# Remove more than one cosecutive space
def remove_extra_spaces(text):
    return re.sub(r'\s+', ' ', text)

# Remove punctuation
df['text_clean'] = df['text'].fillna('').astype(str).apply(lambda x: ''.join(ch for ch in x if ch not in punctuations))

# Remove certain words
for word in words_to_remove:
    regex = re.compile(re.escape(word), re.IGNORECASE)
    df['text_clean'] = df['text_clean'].apply(remove_extra_spaces)
    df['text_clean'] = df['text_clean'].apply(lambda x: regex.sub('', x))

# Remove the text that comes after certain phrases
for phrase in phrases_to_remove_after:
    df['text_clean'] = df['text_clean'].apply(lambda x: x.split(phrase)[0] if phrase in x else x)


# Delete column 'c1' using drop()
df = df.drop('text', axis=1)

# Rename column '
df = df.rename(columns={'text_clean': 'text'})    

# Print the cleaned data
df.to_excel(r'C:\codenrock\clean_output.xlsx', index=False)

In [ ]:
import requests
import time
import pandas as pd
import csv
import re
import json

# Read the Excel file into a pandas DataFrame
df = pd.read_excel(r'C:\clean_output.xlsx')

# Define the base URL for the API
base_url = 'https://x8ki-letl-twmt.n7.xano.io/api:adBtv1oF'

# Define the company parameter
company = 'aero'

request_count = 0  # Counter to keep track of requests

# Prepare columns for the response
df['label'] = None
df['type'] = None
df['json'] = None

for idx, row in df.iterrows():
    email = row['text']
        
    #too long email truncate
    if len(email) > 1000:
        email = email[:700]

    # Define the parameters for the GET request
    params = {'company': company, 'email': email}
        
    # Send the GET request
    response = requests.get(base_url + '/msgp', params=params)
        
    # Check the status code of the response
    if response.status_code == 200:
        print(f"Success! {request_count}")
        try:
            response_json = response.json()
            #request 0:1
            df.at[idx, 'label'] = response_json.get('marker')
            #request type: plane, hotel, train, other
            df.at[idx, 'type'] = response_json.get('type')
            #request json
            df.at[idx, 'json'] =  re.sub(r'\s{2,}','',(re.sub(r'\n', '', response_json.get('json'))))
        except Exception:
            df.at[idx, 'label'] = '0'
    else:
        print(f"Failed! {row['id']}")
        print('Status code:', response.status_code)
        print('Response:', response.text)
        
    request_count += 1
  

# Save the updated DataFrame to the Excel & CVS file
df.to_excel(r'C:\codenrock\result_xls.xlsx', index=False)
df.to_csv(r'C:\codenrock\result_csv.csv', sep=',',  index=False)